In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm
import sys
sys.path.insert(0,'../')
%load_ext autoreload
%autoreload 2
from utils import citibike_helpers
import numpy as np
import pandas as pd  #requirement comes with anaconda
import datetime 
from datetime import datetime as dt
import seaborn as sns
import operator
import networkx as nx
import community #requires separate install -  pip install python-louvain
import warnings
warnings.filterwarnings('ignore')

In [2]:
inputfile="../datasets/citibike/201701-citibike-tripdata.csv.gz"
df=citibike_helpers.load_citibike_data(inputfile)
df.head(2)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,680,2017-01-01 00:00:21,2017-01-01 00:11:41,3226,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,40.775794,-73.976206,25542,Subscriber,1965.0,2
1,1282,2017-01-01 00:00:45,2017-01-01 00:22:08,3263,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,40.748549,-73.988084,21136,Subscriber,1987.0,2


In [3]:
unique_start_stations=citibike_helpers.get_unique_column_values(df,'Start Station ID')
unique_end_stations=citibike_helpers.get_unique_column_values(df,'End Station ID')
station_ids=set(unique_start_stations).union(set(unique_end_stations))
print(unique_start_stations.shape)
print(unique_end_stations.shape)
print("Stations that no one ever starts at but they only end at",set(unique_end_stations).difference(set(unique_start_stations)))

(609,)
(612,)
Stations that no one ever starts at but they only end at {3250, 3447, 3183}


In [4]:
inputfile="../datasets/citibike/201701-citibike-tripdata.csv.gz"
df = citibike_helpers.load_citibike_data(inputfile)
df = citibike_helpers.calculate_trip_durations_citibike(df)
#Here we focus on a subset of "interesting" trips with duration between 5 and 120 minutes.
subset_trips=df[(df['Trip Duration Minutes']>5) & (df['Trip Duration Minutes']<120)]

In [5]:
edges_with_weights=citibike_helpers.infer_weighted_station_station_network(subset_trips)
_thr=0.005
g=citibike_helpers.create_subset_graph(edges_with_weights,thr=_thr,graphtype='Directed')
fig_path="figures/citibike_top_"+str(_thr*100)+"_stn_stn_network.png"
citibike_helpers.plot_network(g, node_dist=0.001, title="",edgealpha=0.3,nodesize=100,savefig=True,filename=fig_path)

TypeError: plot_network() missing 1 required positional argument: 'node_dist'

In [ ]:
print(nx.info(g.to_undirected()))

# Laplacian Matrix
First 8 rows and columns of laplacian matrix for the graph. Please note that, we round the values in the Laplacian matrix just for demonstration purposes, you should not round these values in real world.

In [ ]:
und_g = g.to_undirected()
lpm = nx.laplacian_matrix(und_g)
print(np.round(lpm,2)[0:8,0:8].todense()) 

# Modularity Matrix
First 8 rows and columns of laplacian matrix for the graph. Please note that, we round the values in the Laplacian matrix just for demonstration purposes, you should not round these values in real world.

In [ ]:
modularity_mat = nx.directed_modularity_matrix(g)
print(np.round(modularity_mat,2)[0:8,0:8])

In [ ]:
modulariy_ev = sorted(nx.linalg.spectrum.modularity_spectrum(g),reverse=True)
laplacian_ev = sorted(nx.linalg.spectrum.laplacian_spectrum(und_g),reverse=True)

In [ ]:
print('top 10 eigenvalues for modularity matrix and laplacian matrix:')
for k,v in zip(modulariy_ev[0:10],laplacian_ev[0:10]):
    print('modularity matrix: {}\tlaplacian matrix: {}'.format(k,v))

# Modularity value

In [ ]:
part = community.best_partition(und_g)
print(community.modularity(part, und_g))

# Connected Components

In [ ]:
sub_graphs = nx.connected_component_subgraphs(und_g)
print('number of connected components: ' + str(len([k for k in sub_graphs])))

# Giant Connected Component

In [ ]:
Gc = max(nx.connected_component_subgraphs(und_g), key=len)
Gc=nx.convert_node_labels_to_integers(Gc)
Gc.name='GCC'
print(nx.info(Gc))

# Community Detection
We find the communities for the Giant Connected Component

In [ ]:
plt.figure(figsize=(9,6))
pos = nx.spring_layout(Gc,iterations=200)
partition = community.best_partition(Gc)
values = [partition.get(node) for node in Gc.nodes()]
plt.axis("off")
plt.title('Plot of {} Communities for the Citibike Network'.format(len(set(partition.values()))))
nx.draw_networkx(Gc, pos = pos, cmap = plt.get_cmap("jet"), node_color = values, node_size = 50, with_labels = False)

# Spectral Clustering: Binary Clustering
In binary clustering, we just use the second smallest Eigenvector to decide the labels of nodes.

In [ ]:
L = nx.laplacian_matrix(Gc)
eig_values, eig_vectors = np.linalg.eigh(L.todense()) # Eigen values sorted from smallest to biggest
f = eig_vectors[:,1] # use the second smallest Eigen vector for spectral clustering
labels = np.ravel(np.sign(f)) # decides the label of each node
fig = plt.figure(figsize=(9,6))
plt.axis("off")
nx.draw_networkx(Gc, pos,node_size=45, cmap = plt.get_cmap("jet"), node_color=labels, with_labels = False)

# Spectral Clustering: Multiple Clusters
To find more than 2 clusters, we can use all the Eigenvectors (except the first one) of the Laplacian matrix. Please note that these eigenvectors are sorted based on their respective eigenvalues

In [ ]:
import scipy.cluster.vq as vq
k=4
means, labels = vq.kmeans2(eig_vectors[:,1:k], k)

In [ ]:
fig = plt.figure(figsize=(9,6))
plt.axis("off")
nx.draw_networkx(Gc, pos, node_size=45, cmap = plt.get_cmap("jet"), node_color=labels, with_labels = False)

# Agglomerative Hierarchical Clustering (AHC)
We use complete link (Creates AHC using farthest point linkage) for this example.

# LVL 1: Original Graph

In [ ]:
fig=plt.figure(figsize=(9,6))
plt.axis("off")
nx.draw_networkx(Gc, pos, node_size=45, with_labels = False)
print('original graph (lvl 1) info:')
print(nx.info(Gc))

# AHC: level by level clustering

In [ ]:
H= Gc.copy()
lvl = 2
while(H.number_of_nodes()>1):
    H, partitions = citibike_helpers.draw_hc(H, lvl)
    print('LVL {} cluster info:'.format(lvl))
    print(nx.info(H))
    print('sample of partitions:')
    print(partitions[0:5])
    lvl +=1

# Divisive Hierarchial Clustering

In [ ]:
partition_graph, partitions = citibike_helpers.dhc(Gc)

In [ ]:
lvl = 2
PG = partition_graph[lvl]
partition = partitions[lvl]
labels = np.ones(PG.number_of_nodes())
for i,p in enumerate(partition):
    for k in p:
        labels[k]=int(i)

In [ ]:
citibike_helpers.plot_dhc(PG, partition, labels, lvl, pos)